# <mark> <b> > 1. </b> Mapeamento e analise do  Pipeline </mark>

<b>1_map_analise_pipeline_v0.ipynb </b> |  Atual notebook com as funçoes de mapeamento e analise do pipeline

### Modules & config

In [1]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle



# Modulos da solucao
import modules.cronometro as cron


In [38]:
#### Config - Mapeamento e Analise do pipeline

# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'

# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"


# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"



#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

### Funcoes para Analise do pipeline

In [14]:
# 1. Geraçao do hash do arquivo
def generate_file_hash(file_path):
    # Abre o arquivo em modo de leitura de bytes
    with open(file_path, "rb") as f:
        # Lê o conteúdo do arquivo
        file_data = f.read()
        # Utiliza o algoritmo SHA-256 para gerar o hash
        file_hash = hashlib.sha256(file_data).hexdigest()
    return file_hash



# 2. Geraçao do Unique_id do arquivo
def generate_unique_id():
    return str(uuid.uuid4())


def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result


# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id




# 3. XXX Busca proximo Batch
def busca_proximo_batch():
    # Abre o arquivo Excel e lê a coluna 'batch'
    df = pd.read_excel(conf_export_plan_path, usecols=["batch"])
    # Pega o último valor da coluna 'batch'
    last_value = df.iloc[-1, 0]
    
    # Extraí o número do último batch e adiciona 1 para o próximo
    last_number = int(last_value.split("_")[1])
    next_number = last_number + 1
    
    # Forma o nome do próximo batch
    next_batch = f"Batch_{next_number}"
    
    return next_batch    

# 4. Função para verificar e criar a pasta se não existir
def check_and_create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
# 7. funçao que MOVE documentos e gera add_log_transaction_entry para df_log_transctions
def move_doc_processed_file(batch_name, src_path, tgt_path):
    
    function = "move_doc_processed_file"
    source_path = src_path
    file = os.path.basename(source_path)
    sub_dir = os.path.join(tgt_path, batch_name)
    destination_path = os.path.join(sub_dir, file)
    document_action = "move_processed_file"
    transaction_detail = (f'document {file} moved by: {function}')
    df_move = pd.DataFrame()
    try:
        document_unique_id = get_document_id_by_file(batch_name, file)
        check_and_create_folder(destination_path)
        shutil.move(source_path, destination_path)
        sucess = True
        move_log = add_log_transaction_entry(document_unique_id, batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess)
    except Exception as e:
        print(f"Erro ao mover documento: {e}")
        sucess = False
    
    return move_log   




# 2. XXX Analisa nro de paginas
def analisa_nro_pages(file_path):
    
    pdf_document = fitz.open(file_path)
    pages = pdf_document.pages() # generator object

    page_nro = []
    for page in pages:
        page_nro.append(page)
        
    nro_paginas = len(page_nro)    
    if nro_paginas > 1:
        doc_1_page = False
        return doc_1_page, nro_paginas    
    else:
        doc_1_page = True
        return doc_1_page, nro_paginas  
    pdf_document.close()
    
 

# XXX FUNCAO DE SPLIT
def split_documentos(qualquer_df, fase, atividade, status):
    
    documentos_splitados = []
    doc_info = {}
    rows_list = []
    documentos = []
    #output_dir = os.path.join(documentos_scan_path, batch_name)
    num_linhas_df = qualquer_df.shape[0]

    i = num_linhas_df + 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        nun_pages = row['pages']
        batch_name = row['batch']
        original_file_name = row['original_file_name']
        folder_name = row['directory']
        file_path = row['file_path']
        level = row['level']
        document_type = row['document_type']
        doc_action = row['doc_action']
        document_unique_id = idx
        new_level = level + 1
        
        if (doc_action == 'splitar') and (status == 'root_analise'):
            if nun_pages > 1:
                try:
                    pdf = fitz.open(file_path)
                    # Número total de páginas no PDF
                    total_pages = len(pdf)
                except Exception as e:
                    print(f"Nao congui abrir o PDF: {e}")    

                # Nome base para os arquivos de saída
                base_name = file_path.split('.')[0]  # Remove a extensão do arquivo
                file_to_delete = file_path
                # Loop para criar um novo PDF para cada página
                for page_num in range(total_pages):
                    # Cria um novo objeto PDF
                    new_pdf = fitz.open()
                    # Adiciona a página atual ao novo PDF
                    new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
                    # Nome do novo arquivo PDF
                    new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
                    # Salva o novo PDF
                    new_pdf.save(new_pdf_name)
                    # Fecha o novo PDF
                    new_pdf.close()
                    rotulo = "prov_nota_fiscal"
                    acao_sugerida = sugestoes_acao.get(rotulo, "no_defined_action")
                    acao_executada = "novo_doc_criado"
                    informations = (f'documento criado a partir do split do documento: {original_file_name}')  
                    name_pdf_splited = os.path.basename(new_pdf_name)

                    new_row = {
                        "seq": i,
                        "date_time": cron.timenow_pt_BR(),
                        "batch": batch_name,
                        "fase_processo": fase,
                        "nome_atividade": atividade,
                        "status_documento": status,
                        "acao_executada": acao_executada,
                        "original_file_name": new_pdf_name,
                        "directory": folder_name,
                        "one_page": True,
                        "pages": 1,
                        "document_type": rotulo,
                        "doc_action": acao_sugerida,
                        "level": level,
                        "document_unique_id": generate_unique_id(),
                        "parent_document_unique_id": document_unique_id,
                        "file_hash": generate_file_hash(file_path),
                        "file_path": file_path,
                        "informations": informations,
                    }
                    rows_list.append(new_row)
                    i += 1
                qualquer_df.loc[idx, 'status_documento'] = "NAO_PROCESSAR" 
                qualquer_df.loc[idx, 'informations'] = "Paginas splitada em multiplos documentos" 
                qualquer_df.loc[idx, 'date_time'] = cron.timenow_pt_BR() 
    
    total_split = i - 1        
    df_split = pd.DataFrame(rows_list)
    
    
    return df_split, rows_list


# XXX Usando na criacao da imagem 
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 



def apagar_zone(documentos_extracao_path):
    # Para apagar arquivos PDF:Zone
    for root, dirs, files in os.walk(documentos_extracao_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            #print(file)
            if ":Zone" in file:
                file_to_delete = file_path
                os.remove(file_to_delete)
                #print(file, "termina, pode eliminar")
                
                
def confirma_pdf_pequisavel(file_path):
    
    pdf_document = fitz.open(file_path)
    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    # Definir retângulo de interesse
    try:
        x0 = 0
        y0 = 4
        x1 = 600
        y1 = 200  # Ajuste este valor para delimitar a região vertical
        # Extrair texto dentro do retângulo
        text = page.get_text("text", clip=(x0, y0, x1, y1))
        if text:
            page_number = 0
            pdf_pequisavel = True
        #print(page_number)
        else:
            page_number = 1
            pdf_pequisavel = False
        #print(page_number)
    except Exception as e:
        msg_error = (f"Erro ao abrir pagina do PDF: {e}")
        pdf_pequisavel = False
        pdf_document.close()   
         
        return pdf_pequisavel
                   








def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
    
    # Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "NO_PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)
#print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

## <mark> <b> 3.0</b>  Mapeamento e Analise do pipeline </mark>

In [3]:
# SOMENTE PARA TESTE DE FUNCAO
fake_parent_document_unique_id = 'f976c128-1f41-4551-bffd-fac687c1c8b2'

# Busca proximo Batch caso nao esteja rodando email
batch_name = busca_proximo_batch()
batch_name

In [15]:
# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos - ESSA E A UNICA FUNCAO QUE ITERA NO DIRETORIO
def scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status):
    
    doc_info = {}
    resumo = {}
    raw_document = []
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            nome_arquivo = file
            palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)
            acao_executada = "Analise"
            informations = ' '    
            file_name = file.lower()    
            file_path = os.path.join(root, file)
            new_path_name = os.path.join(output_dir, file)
            if file.lower().endswith('.pdf'):
                doc_one_page, nro_pgs = analisa_nro_pages(file_path)
                one_page = doc_one_page
                num_page = nro_pgs
            else:
                one_page = False
                num_page = 0    
            # if doc_one_page:
            #     one_page = doc_one_page
            # else:
            #     one_page = False    
            #             rotulo = 'pdf_mul_paginas'
            level = 3
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
                
            #print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    
            
            new_row = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "batch": batch_name,
                "fase_processo": fase,
                "nome_atividade": atividade,
                "status_documento": status,
                "acao_executada": acao_executada,
                "original_file_name": file,
                "directory": folder_name,
                "one_page": one_page,
                "pages": num_page,
                "palavra_chave": palavra_chave,
                "document_tag": rotulo,
                "action_item": acao_sugerida,
                "level": level,
                "document_unique_id": generate_unique_id(),
                "parent_document_unique_id": fake_parent_document_unique_id,
                "file_hash": generate_file_hash(file_path),
                "file_path": file_path,
                "informations": informations,
            }
            raw_document.append(new_row)

            
            # print(f'seq: {i} | file: {file}'
            i += 1
    df_trans_pipe = pd.DataFrame(raw_document)
      
                
    return df_trans_pipe, raw_document

In [30]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'root_analise'

documentos = []

#df_trans_pipe = pd.DataFrame()

df_root_pipe, documentos = scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status)


# 0. Acertando o Index
df_root_pipe.set_index('document_unique_id', inplace=True)
df_root_pipe

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,pages,palavra_chave,document_tag,action_item,level,parent_document_unique_id,file_hash,file_path,informations
document_unique_id,,,,,,,,,,,,,,,,,,,
c4dc0fca-4edf-4547-96e3-26790aae2549,1,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,NF CRJ PRIMEIRA QUINZENA DE JULHO DE 2023.pdf,11756286,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,b3acfffea4847108d0064ecbd62a73359961f88741a037...,pipeline_extracao_documentos/2_documentos_para...,
1789136f-6c11-4fe8-a27b-0e38714bbddf,2,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,Nota Fiscal Eletrônica Quallit 24072023.pdf,11766341,True,1,nota,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,ae37292a66dff093838f3cde0da8bea332ec81241b1d5f...,pipeline_extracao_documentos/2_documentos_para...,
5bed1512-08fc-4397-8a5c-90fe3b8c97f0,3,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,NFE 20237.pdf,11624359,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,1976ffe84e27b6bb1a5840018c4a47f6bac483b10f686a...,pipeline_extracao_documentos/2_documentos_para...,
db839bcc-55b5-4890-8259-01c447b7b5b5,4,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,Ampla.pdf,11777624,True,1,default,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,ad2522b53349fffd748376bca7fe4b90fd59359971d992...,pipeline_extracao_documentos/2_documentos_para...,
f41c0878-079b-4520-9168-b376391f2e85,5,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,Supernova.pdf,11777624,True,1,default,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,c26954dad71d508d4b5315f69dc4c6291a1c5bf70548e7...,pipeline_extracao_documentos/2_documentos_para...,
d4ea208e-caaf-41a6-924c-01b02ed0d147,6,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,Blue Lord.pdf,11777624,True,1,default,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,63b6603be7b1df359e67b96e9e128336b46fa3fbcfdadc...,pipeline_extracao_documentos/2_documentos_para...,
ff641182-55a6-496c-be94-d36c212a8518,7,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,NF 2023158.pdf,11285853,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,abcfcce04cabd261b30730f04cbb98e030f2a765bb31aa...,pipeline_extracao_documentos/2_documentos_para...,
b5636ddc-4bb8-4e8a-a0de-867548fd50a5,8,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,NF 2023157.pdf,11285853,True,1,NF,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,c8c5ae8ccca5e9befb6445e21701dc99266746e877c630...,pipeline_extracao_documentos/2_documentos_para...,
9bc6aaf8-e05a-49d6-9642-8f44eb69ea95,9,23/09/2023 12:52:11,Batch_21,analise,scan_analise,root_analise,Analise,nota_07_2023.pdf,11778425,True,1,nota,prov_nota_fiscal,NO_PROCESS,3,f976c128-1f41-4551-bffd-fac687c1c8b2,33bc08587f5c227ab3e035011ed9b20340f99599e65ff7...,pipeline_extracao_documentos/2_documentos_para...,


### Manipulando od DF

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')

In [ ]:
# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_root_pipe[df_root_pipe['one_page'] == False]

In [ ]:
# 0. XXX SE deseja importar o DF df_analise_pipe

df_analise_pipe_path = "Em_HOLD/df_mapeamento_e_analise2.xlsx"


#Le a planilha e cria df_documento_recebido
df_analise_pipe = pd.read_excel(df_analise_pipe_path)

# Ajusta o indice
df_analise_pipe.set_index('document_unique_id', inplace=True)

In [ ]:
# 2B. XXX Efetuo a analise do pipeline de documentos e inicio da extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'Reavaliar_PDF_Pesquisavel' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'mapear'

imagens_list = analise_extracao_pipeline(subset_df_analise_pipe, df_analise_pipe, fase, atividade, status)


if imagens_list:
    remove_images(imagens_list)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

### exportando o map_analise_path

In [40]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_root_pipe
df_root_pipe.to_excel(file_path_root_pipe, index=True)

## RESERVA

In [2]:
 

# 8. Função para adicionar um novo registro em df_source
def add_source_entry(batch_name, file_path, file, type, level, parent_unique_id):
    #unique_id = generate_unique_id(type)
    unique_id = generate_unique_id()
    time_now = cron.timenow_pt_BR()   
    file_hash = generate_file_hash(file_path) 
    if level == 1:
        parent_unique_id = unique_id
    data = {
        'Batch': batch_name,
        'Data': time_now,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_ID': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }
    return data

# 9. Add nova linha para atualizar df_log_transctions
def add_log_transaction_entry(document_unique_id,batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess=True):

    data_log = {
        'Dt_Time': cron.timenow_pt_BR(),
        'Batch': batch_name,
        'File' : file,
        'Unique_ID': document_unique_id,
        'Action': document_action,
        'Sorce': src_path,
        'Target': tgt_path,
        'Transction_Detail': transaction_detail,
        'Sucess': sucess,    
    }
    
        
    return data_log


# 10. Consulta multiparametros
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result




# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id


# 12. Busca filhos - simples
def get_children(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Parent_Unique_ID=document_unique_id)


# 13. Busca pai -simples
def get_father(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    parent_unique_id = result['Parent_Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Unique_ID=parent_unique_id)


# 14. Pesquiso pai pelo Unique_ID e trago um dict
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }


# 15. Pesquiso pai pelo Unique_ID (document_parent_unique_id) e cria DICT
def get_father_by_unique_id(batch, document_parent_unique_id):
    
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=document_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }
    
# 16. Pesquiso pai pelo file do filho e cria DICT
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }        
        

# 17. Busca o 'Unique_ID' para definir o Parent_Unique_ID sem considerar 'Level'
def get_parent_unique_id(df_id_relations, batch_name, file, type):
    try:
        parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type)]['Unique_ID'].values[0]
    except IndexError:
        parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return parent_unique_id


# 18. funcao para trazer somente o 'Unique_ID'
def get_document_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Unique_ID'].values[0]
    except IndexError:
        document_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_unique_id


# 19. funcao para trazer somente o 'Parent_Unique_ID'
def get_document_parent_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Parent_Unique_ID'].values[0]
    except IndexError:
        document_parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_parent_unique_id


# 20. funçao para trazer toda a row de df_id_relations para o documento
def get_document_id_relations(df_id_relations, batch_name, file, type, level):
    try:
        document_id_relations = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)].values[0]
    except IndexError:
        document_id_relations = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_id_relations




# EXEMPLOS de Pesquisa DFss
    # get_document_unique_id(df_id_relations, batch_name, file, type, level)

    # # Busca somente o 'Parent_Unique_ID'
    # get_document_parent_unique_id(df_id_relations, batch_name, file, type, level)


    # #Busca todos os dados da row do documento encontrado
    # document_id_relations = get_document_id_relations(df_id_relations, batch_name, file, type, level)

    # document_batch = document_id_relations[0]
    # document_date = document_id_relations[1]
    # document_name = document_id_relations[2]
    # document_type = document_id_relations[3]
    # document_level = document_id_relations[4]
    # document_unique_id = document_id_relations[5]
    # document_parent_unique_id = document_id_relations[6]
    # document_hash = document_id_relations[7]
    # document_path = document_id_relations[8]

    # # Insercao de um registro pela func add_source_entry
    # file_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/SPA 15082023.rar"

    # file = os.path.basename(file_path)

    # type = "compressed_file_attachment"

    # level = 1

    # parent_unique_id = ''

    # # Adicionando um novo registro (substitua 'batch_name' e 'email' conforme necessário)
    # new_entry = add_source_entry(batch_name, file_path, file, type, level, parent_unique_id)

    # df_id_relations = df_id_relations.append(new_entry, ignore_index=True)

    # df_id_relations




# 1. Criaçao do DataFrame para armazenar as relações de Unique_ID e Parent_Unique_ID
df_id_relations = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'Parent_Unique_ID', 'Hash', 'File_Path'])

# 2. Criaçao do DataFrame para df_start_pipe:
#df_start_pipe = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'dt_hora', 'de', 'assunto', 'email', 'Hash'])

def get_template_version(model):
    row_frame = filtrar_df(frames_nf_v4_df, model=model)
    if not row_frame.empty:
            # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
            version = [((row_frame.iloc[0]['version']))]
            
    return version[0]  




def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result








# XXXpara buscar melhor as coordendas dos FRAMES
def get_coordinates_filter(pdf_pesquisavel_map, model, tipo, label, section):
    
    row_frame = filtrar_df(frames_nf_v4_df, model=model, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates










#generated_parent_document_unique_id = generate_unique_id()  

# Processo de deleçao e atualizacao de documentos
#e_deleta_peloamor(df_docs_splitados)

#me_atualiza_logo_vai_2(novo_df)

# apagar_zone(documentos_extracao_path)


# 5. XXX Ajusta textoYYY
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

# 6. XXX Ajusta texto para PDF_Pesquisavel-  NO CABECALHO
def texto_extraido_nf(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


# 6. XXX Ajusta texto para PDF RASTER NO CABECALHO
def texto_extraido_cabecalho(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    text_splited = [s.replace(")", "").strip() for s in text_splited] #depende da situaçao
    return text_splited
    



#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if index + 1 < len(text_list) and text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return None

    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return None
    
    
    
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return None
        else:
            return None
        

#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    else:
        return None  




def pesquisa_keyword(string_pesquisa, text_splited, keyword_list):
    resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)

    if resultado_extraido_fuzz == None:
        resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
        if resultado_extraido_frame_up == None:
            resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
            resultado_extraido = resultado_extraido_frame_down
        else:
            resultado_extraido = resultado_extraido_frame_up
    else:
        resultado_extraido = resultado_extraido_fuzz

    # Verifica se o resultado extraído é uma das palavras-chave, indicando um erro
    if resultado_extraido in keyword_list:
        resultado_extraido = None

    return resultado_extraido




# XXX Pequenos mas poderosos
def extract_text_PIL(image, coordinates):
    x0, y0, x1, y1 = coordinates
    image_croped = image.crop((x0, y0, x1, y1))
    texto_extraido = pytesseract.image_to_string(image_croped, lang='por', config='--psm 6')
    return texto_extraido 


# 10. Consulta multiparametros
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id


# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None

# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None
    

def corrigir_email(texto):
    # Padrão de regex para identificar e-mails potenciais
    padrao_email = re.compile(r'[a-zA-Z0-9_.+-]+[)!Q@][a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
    
    # Encontrar todos os padrões que se assemelham a um e-mail
    possiveis_emails = padrao_email.findall(texto)
    
    for email in possiveis_emails:
        # Se "@" não estiver presente, tentamos corrigir substituindo ")" ou "!" por "@"
        if "@" not in email:
            email_corrigido = email.replace(")", "@").replace("Q", "@")
            texto = texto.replace(email, email_corrigido)
    
    return texto



# 1 XXX Crio o DF  cnae_x_item_servico_df
cnae_x_item_servico_df = pd.read_excel(cnae_dict_path)

# Mapeando prefeitura e CNAE para a descrição do CNAE
cnae_dict = dict(zip(zip(cnae_x_item_servico_df['PREFE'], cnae_x_item_servico_df['CNA_NUMERO']), cnae_x_item_servico_df['CNA_NOME']))

# Mapeando prefeitura e item de serviço para a descrição do item de serviço e o CNAE associado
item_servico_dict = dict(zip(zip(cnae_x_item_servico_df['PREFE'], cnae_x_item_servico_df['ATV_CODIGO']), zip(cnae_x_item_servico_df['ATV_DESCRICAO'], cnae_x_item_servico_df['CNA_NUMERO'])))


# 2. XXX  Tratando o CNAE com dict criado
def processa_cnae_dict(Texto_extraido, de_para_pm, debug):

    text_splited = Texto_extraido.split('\n')
    # Processando CNAE
    cnae_lines = [line for line in text_splited if 'CNAE' in line]

    if cnae_lines:
        cnae_line = cnae_lines[0]
        #print(f'cnae_line: {cnae_line}')
        
        cnae_number = int(extract_number(cnae_line))
        
        cnae_value = cnae_dict.get((de_para_pm, cnae_number),("Valor não encontrado"))
        if cnae_value != "Valor não encontrado":
            cnae_value = cnae_value.upper()
            cnae_value = str(cnae_number) + " - " + cnae_value
            return cnae_value
        else:
            return None
    else:
        cnae_value = processa_cnae_outros(Texto_extraido)
        cnae_number = int(extract_number(cnae_value))

        cnae_value = cnae_dict.get((de_para_pm, cnae_number),("Valor não encontrado"))
        if cnae_value != "Valor não encontrado":
            cnae_value = cnae_value.upper()
            cnae_value = str(cnae_number) + " - " + cnae_value
            return cnae_value
        else:
            return None

     

# 3. XXX  Tratando Item de Servico com dict criado
def processa_itens_servico_dict(Texto_extraido, de_para_pm, debug):
    
    text_splited = Texto_extraido.split('\n')
    # Encontrando a linha que contém o texto desejado
    item_servico_lines = [line for line in text_splited if 'Item da Lista de Serviços' in line]
    #print(f'item_servico_lines (fora do if): {item_servico_lines}')
    # Verificando se encontramos uma linha válida
    if item_servico_lines:
        #print(f'item_servico_lines: {item_servico_lines}')
        item_servico_line = item_servico_lines[0]
        item_servico_cod = float(extract_number(item_servico_line))
        item_servico, cnae_associado = item_servico_dict.get((de_para_pm, item_servico_cod), ("Valor não encontrado", None))
        item_servico = item_servico.upper()
        item_servico_value = str(item_servico_cod) + " - " + item_servico
        return item_servico_value
    
    else:
        #print("Linha com 'Item da Lista de Serviços' não encontrada")
        item_servico_line = processa_item_sevico_outros(Texto_extraido)
        if item_servico_line:
            item_servico_cod = float(extract_number(item_servico_line))
            item_servico, cnae_associado = item_servico_dict.get((de_para_pm, item_servico_cod), ("Valor não encontrado", None))
            item_servico = item_servico.upper()
            item_servico_value = str(item_servico_cod) + " - " + item_servico
            return item_servico_value
        
        else:
            return None
        #return None






df_id_relations

,Batch,Data,File,Type,Level,Unique_ID,Parent_Unique_ID,Hash,File_Path
